In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('restaurant-comments.xlsx')

In [4]:
df.head()

,comments,date
0,这辈子最爱吃的火锅，一星期必吃一次啊！最近才知道他家还有免费鸡蛋羹………………炒鸡好吃炒鸡嫩...,2017-05-14 16:00:00
1,第N次来了，还是喜欢?……<br><br>从还没上A餐厅的楼梯开始，服务员已经在那迎宾了，然...,2017-05-10 16:00:00
2,大姨过生日，姐姐定的这家A餐厅的包间，服务真的是没得说，A餐厅的服务也是让我由衷的欣赏，很久...,2017-04-20 16:00:00
3,A餐厅的服务哪家店都一样，体贴入微。这家店是我吃过的排队最短的一家，当然也介于工作日且比较晚...,2017-04-25 16:00:00
4,因为下午要去天津站接人，然后我倩前几天就说想吃A餐厅，然后正好这有，就来这吃了。<br>来的...,2017-05-21 16:00:00


注意这里的时间列。如果你的Excel文件里的时间格式跟此处一样，包含了__日期和时间(注意二者缺一不可)__ ，那么Pandas会非常智能地帮你把它识别为时间格式，接着往下做就可以了。
反之，如果你获取到的时间只精确到日期，例如"2017-04-20"这样，那么Pandas只会把它当做字符串，后面的时间序列分析无法使用字符串数据。解决办法是在这里加入以下两行代码：

__from dateutil import parser
df['date'] = df.date.apply(parser.parse)__

In [7]:
text = df.comments.iloc[0]

In [8]:
# 调用SnowNLP情感分析工具
from snownlp import SnowNLP
s = SnowNLP(text)

In [9]:
s.sentiments

0.6331975099099649

In [11]:
def get_sentiment_cn(text):
    s = SnowNLP(text)
    return s.sentiments

df['sentiment'] = df.comments.apply(get_sentiment_cn)

In [13]:
df.head()

,comments,date,sentiment
0,这辈子最爱吃的火锅，一星期必吃一次啊！最近才知道他家还有免费鸡蛋羹………………炒鸡好吃炒鸡嫩...,2017-05-14 16:00:00,0.633198
1,第N次来了，还是喜欢?……<br><br>从还没上A餐厅的楼梯开始，服务员已经在那迎宾了，然...,2017-05-10 16:00:00,0.543631
2,大姨过生日，姐姐定的这家A餐厅的包间，服务真的是没得说，A餐厅的服务也是让我由衷的欣赏，很久...,2017-04-20 16:00:00,1.000000
3,A餐厅的服务哪家店都一样，体贴入微。这家店是我吃过的排队最短的一家，当然也介于工作日且比较晚...,2017-04-25 16:00:00,0.243871
4,因为下午要去天津站接人，然后我倩前几天就说想吃A餐厅，然后正好这有，就来这吃了。<br>来的...,2017-05-21 16:00:00,0.932328


In [14]:
# 求情感分析结果的平均值
df.sentiment.mean()

0.7114015318571119

In [15]:
# 求情感分析结果的中位数
df.sentiment.median()

0.9563139038622388

In [ ]:
# 利用 ggplot 绘图工具包实现可视化
%pylab inline
from ggplot import *

In [ ]:
ggplot(aes(x='date', y='sentiment'), data=df) + geom_point() + geom_line(color = 'blue') + scale_x_date(labels = date_format('%Y-%m-%d'))

In [ ]:
# 利用 Pandas 提供的排序功能可以将所有评论的情感分析得分排序(默认升序排列)
# 下面这行代码显示出了得分最低的评论
df.sort(['sentiment'])[:1]

In [ ]:
# 上面的评论内容显示不全，利用这一行将完整评论打印出来
